In [188]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
import pandas as pd
from imblearn.over_sampling import SMOTENC
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from collections import Counter


In [160]:
# Download latest version
path = kagglehub.dataset_download("rabieelkharoua/alzheimers-disease-dataset")
files = os.listdir(path)
print("Content of", files)

csv_file = files[0]
csv_path = os.path.join(path, csv_file)

# Load DataFrame
df = pd.read_csv(csv_path) 

Content of ['alzheimers_disease_data.csv']


## Data preprocessing

In [161]:
# Remove unnecessary columns
df = df.drop(columns=["PatientID", "DoctorInCharge"], errors="ignore")

# Split data into features (X) and target variable (Y)
X = df.drop(columns=["Diagnosis"])
Y = df["Diagnosis"]

In [146]:
# Sample a subset of the data for analysis
# df = df.sample(frac=1.0, random_state=42)


In [ ]:
# Podział danych na cechy (X) i zmienną celu (Y)
X = df.drop(columns=["Diagnosis"])
Y = df["Diagnosis"]

def change_labels(X):
    custom_labels = {
        'Gender': ['Male', 'Female'],
        'Ethnicity': ['Caucasian', 'African American', 'Asian', 'Other'],
        'EducationLevel': ['None', 'High School', "Bachelor's", 'Higher'],
        'Smoking': ['No', 'Yes'],
        'FamilyHistoryAlzheimers': ['No', 'Yes'],
        'CardiovascularDisease': ['No', 'Yes'],
        'Diabetes': ['No', 'Yes'],
        'Depression': ['No', 'Yes'],
        'HeadInjury': ['No', 'Yes'],
        'Hypertension': ['No', 'Yes'],
        'MemoryComplaints': ['No', 'Yes'],
        'BehavioralProblems': ['No', 'Yes'],
        'Confusion': ['No', 'Yes'],
        'Disorientation': ['No', 'Yes'],
        'PersonalityChanges': ['No', 'Yes'],
        'DifficultyCompletingTasks': ['No', 'Yes'],
        'Forgetfulness': ['No', 'Yes']
    }

    for column, labels in custom_labels.items():
            if column in X.columns:
                # Create a mapping dictionary from numeric values to custom labels
                label_mapping = {i: label for i, label in enumerate(labels)}
                # Replace values in the column using the mapping
                X[column] = X[column].replace(label_mapping)
    return X

X = change_labels(X)
# Podział na zbiory treningowe i testowe
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y
)

In [189]:
# Wyświetlenie proporcji klas przed balansowaniem
print("Proporcje klas przed balansowaniem:")
print(Counter(Y_train))

categorical_features = [index for index, dtype in enumerate(X_train.dtypes) if dtype == "object"]

smote_nc = SMOTENC(categorical_features=categorical_features, random_state=42)

# Balansowanie
X_balanced, Y_balanced = smote_nc.fit_resample(X_train, Y_train)

# Wyświetlenie proporcji klas po balansowaniu
print("Proporcje klas po balansowaniu:")
print(Counter(Y_balanced))

# Finalne dane gotowe do treningu
X_final, Y_final = X_balanced, Y_balanced

Proporcje klas przed balansowaniem:
Counter({0: 1111, 1: 608})


ValueError: SMOTE-NC is not designed to work only with numerical features. It requires some categorical features.

In [186]:
# Dopasowanie modelu Explainable Boosting Classifier
ebm_model = ExplainableBoostingClassifier(random_state=42)
ebm_model.fit(X_final, Y_final)

# Generowanie predykcji i prawdopodobieństw na zbiorze testowym
Y_pred = ebm_model.predict(X_test)
Y_pred_proba = ebm_model.predict_proba(X_test)[:, 1]

print(f"Liczba próbek w Y_test: {len(Y_test)}")
print(f"Liczba próbek w Y_pred: {len(Y_pred)}")

# Ocena modelu
print("EBM - Classification Report:")
print(classification_report(Y_test, Y_pred))
print(f"EBM - AUC ROC: {roc_auc_score(Y_test, Y_pred_proba):.4f}")

/Users/franek/Library/Caches/pypoetry/virtualenvs/alzheimer-s-disease-prediction-oDGSe6K4-py3.13/lib/python3.13/site-packages/interpret/glassbox/_ebm/_ebm.py:813: UserWarning:

Missing values detected. Our visualizations do not currently display missing values. To retain the glassbox nature of the model you need to either set the missing values to an extreme value like -1000 that will be visible on the graphs, or manually examine the missing value score in ebm.term_scores_[term_index][0]



Liczba próbek w Y_test: 430
Liczba próbek w Y_pred: 430
EBM - Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.95      0.95       278
           1       0.90      0.93      0.92       152

    accuracy                           0.94       430
   macro avg       0.93      0.94      0.93       430
weighted avg       0.94      0.94      0.94       430

EBM - AUC ROC: 0.9408


In [172]:
# ebm_local = ebm_model.explain_local(X_test[:5], Y_test[:5])
# show(ebm_local)

ebm_global = ebm_model.explain_global()
show(ebm_global)


<!-- http://127.0.0.1:7001/5303281104/ -->